In [27]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

# Step 1: Load CSV files into Pandas DataFrames
df_structure = pd.read_csv("RefStructure_Old.csv", sep=";")

df_structure


,Cat1EN,Cat2EN,DescriptionCat2EN
0,purchases of services,equipment rental,rental of various types of equipment for tempo...
1,purchases of services,furniture rental,"furniture hire for temporary needs, such as fi..."
2,purchases of services,vehicle rental,"temporary provision of motorised vehicles, suc..."
3,purchases of services,building maintenance and services,"maintenance and repair of buildings, excluding..."
4,purchases of services,building maintenance and services,"maintenance and repair of buildings, excluding..."
...,...,...,...
684,passenger transport,road,"passenger travel on the road network, includin..."
685,passenger transport,road,"passenger travel on the road network, includin..."
686,passenger transport,road,"passenger travel on the road network, includin..."
687,passenger transport,road,"passenger travel on the road network, includin..."


In [24]:
df_TransactionData = pd.read_csv("TransactionData_1000Rows.csv", sep=";")  
df_TransactionData


,Fam1EN,CostTypeEN,BookingEN,ActivityKey
0,other personnel expenses,Travel expenses,zürich berlin visit Oberbuchsiten information ...,Train (Staff)
1,"property insurance, fees, levies",Fees and charges (not for vehicles),zuordbar costs fees customs,Financial services
2,"maintenance, repairs, replacements",Maintenance subscription (without computer),zumtobel maintenance subscription,Technical maintenance
3,external services,DZ - electric material,zumtobel elektrom document licht container,Electrical equipments
4,it expenses,Consumables computer (Hardware),zumstein vision,IT equipments
...,...,...,...,...
994,"energy, operating and auxiliary materials",Water,water iv,Water
995,"energy, operating and auxiliary materials",Water,water iv,Water
996,"energy, operating and auxiliary materials",Water,water iv,Water
997,"energy, operating and auxiliary materials",Water,water iv,Water


In [30]:
pip install setfit transformers datasets


  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9
Note: you may need to restart the kernel to use updated packages.


In [33]:
import pandas as pd
from setfit import SetFitModel
from sentence_transformers import util
import torch

# Step 2: Combine text fields in both dataframes (fill NaNs with empty strings)
df_TransactionData['combined_text'] = df_TransactionData['CostTypeEN'].fillna('') + ' ' + df_TransactionData['BookingEN'].fillna('')
df_structure['Cat1_Cat2_Description_combined'] = df_structure[['Cat1EN', 'Cat2EN', 'DescriptionCat2EN']].fillna('').agg(' '.join, axis=1)

# Step 3: Load the SetFit model for encoding text (this assumes you have the 'setfit-particular-transaction-solon-embeddings-labels-large-kaggle-automatisation-v1' model fine-tuned and ready)
model = SetFitModel.from_pretrained('HEN10/setfit-particular-transaction-solon-embeddings-labels-large-kaggle-automatisation-v1')

# Step 4: Generate embeddings for combined text columns in both dataframes using SetFit
transaction_embeddings = model.encode(df_TransactionData['combined_text'].tolist())
structure_embeddings = model.encode(df_structure['Cat1_Cat2_Description_combined'].tolist())

# Step 5: Calculate cosine similarity between each transaction entry and structure entry
matches = []
for i, transaction_embedding in enumerate(transaction_embeddings):
    cosine_scores = util.pytorch_cos_sim(torch.tensor(transaction_embedding), torch.tensor(structure_embeddings)).flatten()
    best_match_idx = cosine_scores.argmax().item()
    max_score = cosine_scores[best_match_idx].item()
    
    # Collect the matching information
    matches.append({
        'Transaction_Index': i,
        'Cat1_Match': df_structure.iloc[best_match_idx]['Cat1EN'],
        'Cat2_Match': df_structure.iloc[best_match_idx]['Cat2EN'],
        'DescriptionCat2_Match': df_structure.iloc[best_match_idx]['DescriptionCat2EN'],
        'Similarity_Score': max_score
    })

# Step 6: Create a result DataFrame with matches and similarity scores
result_df = pd.DataFrame(matches)

# Step 7: Merge the results with the original transaction data
result_df = pd.merge(df_TransactionData, result_df, left_index=True, right_on='Transaction_Index', how='left')

# Step 8: Save the result to a new CSV file or database table
result_df.to_csv("Matched_Results_with_SetFit.csv", sep=";", index=False)

print("Matching complete using SetFit. Results saved to 'Matched_Results_with_SetFit.csv'.")


c:\Users\yzammit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Matching complete using SetFit. Results saved to 'Matched_Results_with_SetFit.csv'.
